In [1]:
import folium
import folium.plugins
m = folium.Map(arr[0], zoom_start=10, tiles="http://capsule.local:8080/styles/klokantech-basic/{z}/{x}/{y}.png", attr="capsule-maps") 
m

NameError: name 'arr' is not defined

In [6]:
from influxdb import DataFrameClient
import pandas as pd
from datetime import datetime, timedelta

influxdb_client =  DataFrameClient("localhost", "8086", "rudloff", "y4uv3jpc", "telegraf")

#SELECT MEAN("value") FROM "autogen"."mqtt_consumer" 
# WHERE ("topic" = 'gps_measure/latitude') AND time >= '2021-08-05T06:00:00Z'
# AND TIME <= '2021-08-05T06:03:00Z' GROUP BY time(1m) fill(previous)

start = (datetime.strptime("2021-08-05T06:03:00", "%Y-%m-%dT%H:%M:%S") + timedelta(seconds=-5)).isoformat()
end = (datetime.strptime("2021-08-05T07:03:00", "%Y-%m-%dT%H:%M:%S") + timedelta(seconds=5)).isoformat()
print(start)
print(end)
# Query the lat and lon values inside of the first and last + margin timestamps
result = influxdb_client.query("SELECT MEAN(\"value\")  FROM \"autogen\".\"mqtt_consumer\" WHERE (\"topic\"\
     = 'gps_measure/latitude' or \"topic\" = 'gps_measure/longitude') AND time >= '"+start+"Z' AND time <= '"+end+"Z'  GROUP BY time(5s) fill(previous)")
latitude = result["mqtt_consumer"]

latitude

2021-08-05T06:02:55
2021-08-05T07:03:05


,mean
2021-08-05 06:02:55+00:00,25.52880
2021-08-05 06:03:00+00:00,25.52880
2021-08-05 06:03:05+00:00,25.52880
2021-08-05 06:03:10+00:00,25.52880
2021-08-05 06:03:15+00:00,25.52880
...,...
2021-08-05 07:02:45+00:00,25.52935
2021-08-05 07:02:50+00:00,25.52935
2021-08-05 07:02:55+00:00,25.52935
2021-08-05 07:03:00+00:00,25.52935


In [40]:
def parse_influxdb_2_tripdb(timestamps,  influxdb_client: DataFrameClient, resampling_time: str) -> pd.DataFrame():
    start = (datetime.fromisoformat(timestamps[0]) + timedelta(seconds=-5)).isoformat()
    end = (datetime.fromisoformat(timestamps[-1]) + timedelta(seconds=5)).isoformat()
    # Query the lat and lon values inside of the first and last + margin timestamps
    def query(topic: str):    
        result = influxdb_client.query("SELECT MEAN(\"value\") FROM \"autogen\".\"mqtt_consumer\" WHERE (\"topic\"\
            = '"+topic+"') AND time >= '"+start+"Z' AND time <= '"+end+"Z' GROUP BY time("+resampling_time+") fill(previous)")
        res_pd = pd.DataFrame()
        if result:
            res_pd = result["mqtt_consumer"]
        return res_pd
    latitude = query("gps_measure/latitude")
    longitude = query("gps_measure/longitude")
    altitude = query("gps_measure/altitude")
    speed = query("gps_measure/speed")
    #km = query("odometry/km")

    # Clean the values to one specific DataFrame
    results = pd.DataFrame()
    results = latitude.copy()
    results["longitude"] = longitude["mean"]
    results["altitude"] = altitude["mean"]
    results["speed"] = speed["mean"]
    results["km"] = speed["mean"] # TODO replace with km when ready
    results.columns = ["latitude", "longitude", "altitude", "speed", "km"]

    # Filter out only on timestamps
    mask = (results.index >= timestamps[0]) & (results.index <= timestamps[-1])
    return results.loc[mask]
now = datetime.now()
parsed = parse_influxdb_2_tripdb(
    [str(datetime(now.year, now.month, now.day).isoformat()), str(datetime(now.year, now.month, now.day, 23, 55).isoformat())], 
    influxdb_client, "5s")
parsed

DatetimeIndex(['2021-08-06 00:00:00+00:00', '2021-08-06 00:00:05+00:00',
               '2021-08-06 00:00:10+00:00', '2021-08-06 00:00:15+00:00',
               '2021-08-06 00:00:20+00:00', '2021-08-06 00:00:25+00:00',
               '2021-08-06 00:00:30+00:00', '2021-08-06 00:00:35+00:00',
               '2021-08-06 00:00:40+00:00', '2021-08-06 00:00:45+00:00',
               ...
               '2021-08-06 23:54:15+00:00', '2021-08-06 23:54:20+00:00',
               '2021-08-06 23:54:25+00:00', '2021-08-06 23:54:30+00:00',
               '2021-08-06 23:54:35+00:00', '2021-08-06 23:54:40+00:00',
               '2021-08-06 23:54:45+00:00', '2021-08-06 23:54:50+00:00',
               '2021-08-06 23:54:55+00:00', '2021-08-06 23:55:00+00:00'],
              dtype='datetime64[ns, UTC]', length=17221, freq=None)

In [ ]:
latitude = pd.DataFrame()
if result:
    latitude = result["mqtt_consumer"]
result = influxdb_client.query("SELECT * FROM \"autogen\".\"mqtt_consumer\" WHERE (\"topic\"\
     = 'gps_measure/longitude') AND time >= '"+start+"Z' AND time <= '"+end+"Z'")
longitude = pd.DataFrame()
if result:
    longitude = result["mqtt_consumer"]
# Check the values
if latitude.empty or longitude.empty:
    return pd.DataFrame()
# Clean, resample 1s just to be sure, add the values to one specific DataFrame
latitude.drop(["host", "topic"], axis=1, inplace=True)
longitude.drop(["host", "topic"], axis=1, inplace=True)
latitude = latitude.resample('1S').first()
longitude = longitude.resample('1S').first()
gps_coords = pd.DataFrame()
gps_coords = latitude.copy()
gps_coords["longitude"] = longitude["value"]
gps_coords.columns = ["latitude", "longitude"]
# Fill NaN cells
gps_coords = gps_coords.fillna(method="ffill")
# Filter out only on timestamps from the images
mask = (gps_coords.index >= timestamps[0]) & (gps_coords.index <= timestamps[-1])
gps_coords = gps_coords.loc[mask]

In [ ]:
path_coords = [[,],[,]]

In [ ]:

m2 = folium.Map(arr[1], zoom_start=10)
folium.plugins.AntPath(
    locations=arr, 
    dash_array=[10, 15],
    delay=800,
    weight=6,
    color="#F6FFF3",
    pulse_color="#000000",
    paused=False,
    reverse=False,
).add_to(m2)
m2.fit_bounds(m2.get_bounds())
m2

NameError: name 'arr' is not defined

In [ ]:


m3 = folium.Map(arr[1], zoom_start=10)

folium.plugins.AntPath(
    locations=arr, 
    dash_array=[10, 15],
    delay=800,
    weight=6,
    color="#F6FFF3",
    pulse_color="#000000",
    paused=False,
    reverse=False,
    tooltip='<h1>10 Décembre 2021</h1><p>Etape X</p><p>Distance parcourue X km</p><img src="road.gif">'
).add_to(m3)
m3.fit_bounds(m3.get_bounds())


marker_cluster = MarkerCluster(name='Photos during the trip').add_to(m3)
marker_cluster_steps = FeatureGroup(name="Trip steps").add_to(m3)
marker_unesco_cluster = MarkerCluster(name='Unesco sites').add_to(m3)

encoded = base64.b64encode(open("moon.jpg", 'rb').read())
html = '<img src="data:image/jpeg;base64,{}">'.format
resolution, width, height = 15, 50, 25
iframe = IFrame(html(encoded.decode('UTF-8')), width=(width*resolution)+20, height=(height*resolution)+20)
popup = folium.Popup(iframe, max_height=1000)

kw = {"prefix": "fa", "color": "green", "icon": "camera"}
# Create photos marker
for i in range(0, len(arr), 200):
    loc = list(arr[i])
    loc[0] += random.uniform(-0.5, 0.5)
    loc[1] += random.uniform(-0.5, 0.5)
    
    encoded = base64.b64encode(open("moon.jpg", 'rb').read())
    html = '<h1>10 Décembre 2021</h1><p>Etape X</p><p>Distance parcourue X km</p><p>Coordonnée GPS: {}, {}</p><p><img src="data:image/jpeg;base64,{}"></p>'.format
    icon = folium.Icon(**kw)
    folium.Marker(location=loc, icon=icon, tooltip=html(loc[0], loc[1], encoded.decode('UTF-8'))).add_to(marker_cluster)

kw = {"prefix": "fa", "color": "blue", "icon": "bed"}
# Create sleep steps
for i in range(0, len(arr), 2000):
    loc = list(arr[i])
    html = '<h1>10 Décembre 2021</h1><p>Etape X</p><p>Distance parcourue X km</p><p>Coordonnée GPS: {}, {}</p>'.format(loc[0], loc[1])
    icon = folium.Icon(**kw)
    folium.Marker(location=loc, icon=icon, tooltip=html).add_to(marker_cluster_steps)

kw = {"prefix": "fa", "color": "red", "icon": "university"}

for i in range(0, len(unesco_sites)):
    loc = [unesco_sites["latitude"][i], unesco_sites["longitude"][i]]
    icon = folium.Icon(**kw)
    folium.Marker(location=loc, icon=icon, popup=folium.Popup(unesco_sites["short_description_fr"][i], max_width=1000), tooltip=unesco_sites["name_fr"][i]+" \n click for more details").add_to(marker_unesco_cluster)
    
    
legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 10px;
    left: 10px;
    width: 250px;
    height: 100px;
    z-index:9999;
    font-size:14px;
    ">
    <p>Temps de voyage: 54 jours</p>
    <p>Kilomètre parcourue: 13141 km</p>
    <p>Pays traversés: 5</p>
    <p>Date de mise à jour: 10 mars 2021</p>
</div>
<div style="
    position: fixed;
    bottom: 0px;
    left: 0px;
    width: 250px;
    height: 120px;
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;
    opacity: 0.7;
    ">
</div>
{% endmacro %}
"""

legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)
m3.get_root().add_child(legend)


folium.LayerControl(collapsed=False).add_to(m3)

plugins.LocateControl().add_to(m3)

plugins.Fullscreen(
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m3)

m3.save("test.html")

m3

NameError: name 'folium' is not defined